Load in the data and explore

In [1]:
import pandas as pd
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
print(data.info)

<bound method DataFrame.info of       RowNumber  CustomerId    Surname  CreditScore Geography  Gender  Age  \
0             1    15634602   Hargrave          619    France  Female   42   
1             2    15647311       Hill          608     Spain  Female   41   
2             3    15619304       Onio          502    France  Female   42   
3             4    15701354       Boni          699    France  Female   39   
4             5    15737888   Mitchell          850     Spain  Female   43   
...         ...         ...        ...          ...       ...     ...  ...   
9995       9996    15606229   Obijiaku          771    France    Male   39   
9996       9997    15569892  Johnstone          516    France    Male   35   
9997       9998    15584532        Liu          709    France  Female   36   
9998       9999    15682355  Sabbatini          772   Germany    Male   42   
9999      10000    15628319     Walker          792    France  Female   28   

      Tenure    Balance  NumOfP

In [4]:
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [5]:
data.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

preprocess the data

In [6]:
#drop irrelavent columns
data = data.drop(columns=['RowNumber','CustomerId','Surname'])
#seperate features and target
x=data.drop(columns=['Exited'])
y=data['Exited']

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#define categorical columns
cat_features=['Geography','Gender']
#create new column to apply one-hot encoding
preprocessor=ColumnTransformer(
    transformers=[
        ('cat',OneHotEncoder(),cat_features)
    ],
    remainder='passthrough'
)

#apply transformation to features
x_encoded = preprocessor.fit_transform(x)

Feature scaling, since it is necessary for neural networks. 

In [9]:
from sklearn.preprocessing import StandardScaler

scaler= StandardScaler()
x_scaled = scaler.fit_transform(x_encoded)

Build and train the ANN model. Creating a three layered ANN model using Keras

In [10]:
#load in necessary functionality
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score

In [11]:
#split the data
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,test_size=0.2, random_state = 42)

Initialize the ANN model and add the three layers

In [12]:
model=Sequential()
model.add(Dense(units=64,activation='relu',input_shape=(x_train.shape[1],)))
model.add(Dense(units=32,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

/Users/connorbetancourt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
#compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Train the model

In [15]:
history= model.fit(x_train,y_train,epochs=10,batch_size=32,validation_split=0.2)

Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - accuracy: 0.7582 - loss: 0.5186 - val_accuracy: 0.8350 - val_loss: 0.4124
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 321us/step - accuracy: 0.8265 - loss: 0.4128 - val_accuracy: 0.8487 - val_loss: 0.3810
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 314us/step - accuracy: 0.8424 - loss: 0.3787 - val_accuracy: 0.8537 - val_loss: 0.3607
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 308us/step - accuracy: 0.8598 - loss: 0.3524 - val_accuracy: 0.8562 - val_loss: 0.3533
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 315us/step - accuracy: 0.8570 - loss: 0.3448 - val_accuracy: 0.8550 - val_loss: 0.3494
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 311us/step - accuracy: 0.8628 - loss: 0.3431 - val_accuracy: 0.8556 - val_loss: 0.3477
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 316us/step - accuracy: 0.8605 - loss: 0.3359 - val_accuracy: 0.8531 - val_loss: 0.3466
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 310us/step - accuracy: 0.8687 - loss: 0.3252 - 

Make predictions

In [16]:
y_pred= model.predict(x_test)
y_pred_binary = (y_pred>0.5).astype(int)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step


Evaluate the model

In [17]:
accuracy = accuracy_score(y_test,y_pred_binary)
conf_matrix = confusion_matrix(y_test,y_pred_binary)

print("accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)

accuracy: 0.8575
Confusion Matrix:
 [[1536   71]
 [ 214  179]]
